In [1]:
#Importo Librerias
import pandas as pd
import glob
import datetime
import numpy as np

In [2]:
#Cargo los archivos separados por ; en un df llamado df_semicolon
path = r'C:\Users\scamacho\Desktop\COVID-19\Historico de Casos Python\puntoycoma' # use your path
all_files_abril = glob.glob(path + "/*.csv")

li = []
n = 0

for filename in all_files_abril:
    df = pd.read_csv(filename, index_col=None,sep=";")
    df['source'] = all_files_abril[n][-21:]
    n = n+1
    li.append(df)

df_semicolon = pd.concat(li, axis=0, ignore_index=True)
all_files_abril[:5]

['C:\\Users\\scamacho\\Desktop\\COVID-19\\Historico de Casos Python\\puntoycoma\\Casos1 - 20-marzo.csv',
 'C:\\Users\\scamacho\\Desktop\\COVID-19\\Historico de Casos Python\\puntoycoma\\Casos1 - 23-marzo.csv',
 'C:\\Users\\scamacho\\Desktop\\COVID-19\\Historico de Casos Python\\puntoycoma\\Casos1 - 25-abril.csv',
 'C:\\Users\\scamacho\\Desktop\\COVID-19\\Historico de Casos Python\\puntoycoma\\Casos1 - 5-abril.csv',
 'C:\\Users\\scamacho\\Desktop\\COVID-19\\Historico de Casos Python\\puntoycoma\\Casos1 - 6-abril.csv']

In [3]:
#Cargo los archivos separados por  en un df llamado df_colon
path = r'C:\Users\scamacho\Desktop\COVID-19\Historico de Casos Python\Con_coma' # use your path
all_files_abril2 = glob.glob(path + "/*.csv")

li = []
n = 0


for filename in all_files_abril2:
    df = pd.read_csv(filename, index_col=None,sep=",")
    df['source'] = all_files_abril2[n][-21:]
    n = n+1
    li.append(df)

df_colon = pd.concat(li, axis=0, ignore_index=True)
       

In [4]:
#Junto los dos df en uno
data = pd.concat([df_semicolon,df_colon], axis=0, ignore_index=True)

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 153051 entries, 0 to 153050
Data columns (total 24 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   ID de caso                153051 non-null  object 
 1   Fecha de diagnóstico      49297 non-null   object 
 2   Ciudad de ubicación       153051 non-null  object 
 3   Atención                  7359 non-null    object 
 4   Edad                      153050 non-null  object 
 5   Sexo                      153051 non-null  object 
 6   Tipo                      114859 non-null  object 
 7   País de procedencia       153040 non-null  object 
 8   source                    153051 non-null  object 
 9   Departamento              6215 non-null    object 
 10  Tipo*                     38192 non-null   object 
 11  atención                  114578 non-null  object 
 12  Codigo DIVIPOLA           107546 non-null  float64
 13  Departamento o Distrito   117839 non-null  o

In [6]:
#Uno todas las columnas de atención
atencion = data.loc[:, ['Atención', 'atención', 'Atención**','source']]
atencion['new'] = atencion['Atención'].fillna('') + atencion['atención'].fillna('') + atencion['Atención**'].fillna('')
#atencion['new'].value_counts()

In [7]:
#Unifico Valores
atencion['new'] = atencion['new'].replace({"recuperado": "Recuperado", "Recuperado (Hospital)": "Recuperado", "hospital": "Hospital", "fallecido" :"Fallecido", "casa":"Casa", "En casa":"Casa", "En Casa":"Casa", "CASA":"Casa", "":"No informa"})
atencion['new'] = atencion['new'].replace({"-   -": "Descartado"})
atencion['new'].value_counts()

Casa            104549
Recuperado       27872
Hospital         10524
Fallecido         6384
Hospital UCI      3654
Descartado          40
No informa          28
Name: new, dtype: int64

In [8]:
#Agrego la columna limpia a los datos y borro las columnas 
data['Lugar_Atencion'] = atencion['new']
data = data.drop(['Atención', 'atención', 'Atención**'], 1)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 153051 entries, 0 to 153050
Data columns (total 22 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   ID de caso                153051 non-null  object 
 1   Fecha de diagnóstico      49297 non-null   object 
 2   Ciudad de ubicación       153051 non-null  object 
 3   Edad                      153050 non-null  object 
 4   Sexo                      153051 non-null  object 
 5   Tipo                      114859 non-null  object 
 6   País de procedencia       153040 non-null  object 
 7   source                    153051 non-null  object 
 8   Departamento              6215 non-null    object 
 9   Tipo*                     38192 non-null   object 
 10  Codigo DIVIPOLA           107546 non-null  float64
 11  Departamento o Distrito   117839 non-null  object 
 12  Estado                    107518 non-null  object 
 13  Fecha de muerte           100460 non-null  o

In [9]:
#Junto las columnas de departamento
depto = data.loc[:, ['Departamento', 'Departamento o Distrito ', 'Departamento o Distrito','source']]
depto['new'] = depto['Departamento'].fillna('') + depto['Departamento o Distrito '].fillna('') + depto['Departamento o Distrito'].fillna('')
depto['new'].value_counts()

Bogotá D.C.         53870
Valle del Cauca     19554
Antioquia           11000
BOGOTÁ D.C.          6418
Meta                 6048
                    ...  
San Andrés Islas        8
SUCRE                   5
LA GUAJIRA              5
AMAZONAS                1
Cundinamara             1
Name: new, Length: 73, dtype: int64

##### PENDIENTE

In [10]:
#Archivos con casos sin departamento
ensayo = depto[depto['new']=='']
ensayo['source'].value_counts()

Casos1 - 22-marzo.csv    231
Casos1 - 21-marzo.csv    196
Casos1 - 20-marzo.csv    145
Casos1 - 19-marzo.csv    108
Name: source, dtype: int64

In [11]:
depto['new'] = depto['new'].str.upper()
depto['new'] = depto['new'].replace({"CARTAGENA": "BOLÍVAR", "SANTA MARTA": "MAGDALENA", "BARRANQUILLA": "ATLÁNTICO", "SAN ANDRÉS" :"ARCHIPIÉLAGO DE SAN ANDRÉS, PROVIDENCIA Y SANTA CATALINA", "ARCHIPIÉLAGO DE SAN ANDRÉS":"ARCHIPIÉLAGO DE SAN ANDRÉS, PROVIDENCIA Y SANTA CATALINA", "SANTA MARTA D.T. Y C.":"MAGDALENA", "CARTAGENA D.T. Y C.":"BOLÍVAR", "CARTAGENA D.T. Y C":"BOLÍVAR", "BOGOTÁ, D.C.":"BOGOTÁ D.C"})
depto['new'] = depto['new'].replace({"BARRANQUILLA D.E.": "ATLÁNTICO", "VALLE": "VALLE DEL CAUCA", "BUENAVENTURA D.E.": "VALLE DEL CAUCA","ARCHIPIÉLAGO DE SAN ANDRÉS PROVIDENCIA Y SANTA CATALINA":"ARCHIPIÉLAGO DE SAN ANDRÉS, PROVIDENCIA Y SANTA CATALINA","BOGOTÁ":"BOGOTÁ D.C.","SAN ANDRÉS ISLAS":"ARCHIPIÉLAGO DE SAN ANDRÉS, PROVIDENCIA Y SANTA CATALINA", "QUINDIO":"QUINDÍO", "CUNDINAMARA":"CUNDINAMARCA","-   -": "Descartado"})
#depto['new'].value_counts()

In [12]:
#Agrego la columna limpia a los datos y borro las columnas 
data['Depto'] = depto['new']
data = data.drop(['Departamento', 'Departamento o Distrito ', 'Departamento o Distrito'], 1)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 153051 entries, 0 to 153050
Data columns (total 20 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   ID de caso             153051 non-null  object 
 1   Fecha de diagnóstico   49297 non-null   object 
 2   Ciudad de ubicación    153051 non-null  object 
 3   Edad                   153050 non-null  object 
 4   Sexo                   153051 non-null  object 
 5   Tipo                   114859 non-null  object 
 6   País de procedencia    153040 non-null  object 
 7   source                 153051 non-null  object 
 8   Tipo*                  38192 non-null   object 
 9   Codigo DIVIPOLA        107546 non-null  float64
 10  Estado                 107518 non-null  object 
 11  Fecha de muerte        100460 non-null  object 
 12  Fecha de notificación  103754 non-null  object 
 13  Fecha diagnostico      107546 non-null  object 
 14  Fecha recuperado       104971 non-nu

In [13]:
#Junto las columnas de tipo
tipo = data.loc[:, ['Tipo', 'Tipo*','source']]
tipo['new'] = tipo["Tipo"].combine_first(tipo["Tipo*"])
#tipo['new'].value_counts()

In [14]:
tipo['new'] = tipo['new'].replace({"En estudio": "En Estudio", "en estudio": "En Estudio", "en Estudio": "En Estudio", "relacionado" :"Relacionado", "importado":"Importado", "-   -": "Descartado", "S.D.":"En estudio"})
tipo['new'].value_counts()

En Estudio     72537
Relacionado    51002
Importado      29463
Descartado        40
En estudio         9
Name: new, dtype: int64

In [15]:
#Agrego la columna limpia a los datos y borro las columnas 
data['Tipo_Arreglado'] = tipo['new']
data = data.drop(['Tipo', 'Tipo*'], 1)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 153051 entries, 0 to 153050
Data columns (total 19 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   ID de caso             153051 non-null  object 
 1   Fecha de diagnóstico   49297 non-null   object 
 2   Ciudad de ubicación    153051 non-null  object 
 3   Edad                   153050 non-null  object 
 4   Sexo                   153051 non-null  object 
 5   País de procedencia    153040 non-null  object 
 6   source                 153051 non-null  object 
 7   Codigo DIVIPOLA        107546 non-null  float64
 8   Estado                 107518 non-null  object 
 9   Fecha de muerte        100460 non-null  object 
 10  Fecha de notificación  103754 non-null  object 
 11  Fecha diagnostico      107546 non-null  object 
 12  Fecha recuperado       104971 non-null  object 
 13  fecha reporte web      95628 non-null   object 
 14  FIS                    111146 non-nu

In [16]:
#Arreglo la columna Estado
data['Estado'] = data['Estado'].replace({"fallecido": "Fallecido", "Moderada": "Moderado", "leve": "Leve", "-   -": "Descartado"})
data['Estado'].value_counts()

Leve            92809
Moderado         6828
Fallecido        4868
Grave            2046
Asintomático      927
Descartado         40
Name: Estado, dtype: int64

In [17]:
#Arreglo la columna Sexo
data['Sexo'] = data['Sexo'].replace({"Hombre": "M", "Mujer": "F", "m": "M", "f" :"F", "-   -": "Descartado", "0" :"F"})
data['Sexo'].value_counts()

M             80269
F             72742
Descartado       40
Name: Sexo, dtype: int64

In [18]:
#Junto las columnas de Fecha de diagnóstico
FD = data.loc[:, ['Fecha de diagnóstico', 'Fecha diagnostico','source']]
FD['new'] = FD["Fecha de diagnóstico"].combine_first(FD["Fecha diagnostico"])
#FD['new'].value_counts()
FD.head(3)

,Fecha de diagnóstico,Fecha diagnostico,source,new
0,3/06/2020,NaN,Casos1 - 20-marzo.csv,3/06/2020
1,3/09/2020,NaN,Casos1 - 20-marzo.csv,3/09/2020
2,3/09/2020,NaN,Casos1 - 20-marzo.csv,3/09/2020


In [19]:
#Agrego la columna y borro las columnas 
data['Fecha Diagnostico'] = FD['new']
data = data.drop(['Fecha de diagnóstico', 'Fecha diagnostico'], 1)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 153051 entries, 0 to 153050
Data columns (total 18 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   ID de caso             153051 non-null  object 
 1   Ciudad de ubicación    153051 non-null  object 
 2   Edad                   153050 non-null  object 
 3   Sexo                   153051 non-null  object 
 4   País de procedencia    153040 non-null  object 
 5   source                 153051 non-null  object 
 6   Codigo DIVIPOLA        107546 non-null  float64
 7   Estado                 107518 non-null  object 
 8   Fecha de muerte        100460 non-null  object 
 9   Fecha de notificación  103754 non-null  object 
 10  Fecha recuperado       104971 non-null  object 
 11  fecha reporte web      95628 non-null   object 
 12  FIS                    111146 non-null  object 
 13  Fecha reporte web      7769 non-null    object 
 14  Lugar_Atencion         153051 non-nu

In [20]:
#Junto las columnas de Fecha de Reporte Web
FRW = data.loc[:, ['fecha reporte web', 'Fecha reporte web','source']]
FRW['new'] = FRW["fecha reporte web"].combine_first(FRW["Fecha reporte web"])

In [21]:
#Agrego la columna y borro las columnas 
data['Fecha_Reporte_Web'] = FRW['new']
data = data.drop(['fecha reporte web', 'Fecha reporte web'], 1)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 153051 entries, 0 to 153050
Data columns (total 17 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   ID de caso             153051 non-null  object 
 1   Ciudad de ubicación    153051 non-null  object 
 2   Edad                   153050 non-null  object 
 3   Sexo                   153051 non-null  object 
 4   País de procedencia    153040 non-null  object 
 5   source                 153051 non-null  object 
 6   Codigo DIVIPOLA        107546 non-null  float64
 7   Estado                 107518 non-null  object 
 8   Fecha de muerte        100460 non-null  object 
 9   Fecha de notificación  103754 non-null  object 
 10  Fecha recuperado       104971 non-null  object 
 11  FIS                    111146 non-null  object 
 12  Lugar_Atencion         153051 non-null  object 
 13  Depto                  153051 non-null  object 
 14  Tipo_Arreglado         153051 non-nu

In [22]:
#Organizo la fecha de los archivos
FA = data['source'].str.split("-", n = 1, expand = True) 
Dia = FA.iloc[:, 1].str.split("-", n = 1, expand = True) 
Mes = Dia.iloc[:, 1].str.split(".", n = 1, expand = True) 

data['dia_archivo'] = Dia.iloc[:, 0]
data['mes_archivo'] = Mes.iloc[:, 0]

data['mes_archivo'] = data['mes_archivo'].replace({"enero":"01","febrero":"02","marzo":"03","abril":"04","mayo":"05","junio":"06","julio":"07"})
data['año'] = '2020'

In [23]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 153051 entries, 0 to 153050
Data columns (total 20 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   ID de caso             153051 non-null  object 
 1   Ciudad de ubicación    153051 non-null  object 
 2   Edad                   153050 non-null  object 
 3   Sexo                   153051 non-null  object 
 4   País de procedencia    153040 non-null  object 
 5   source                 153051 non-null  object 
 6   Codigo DIVIPOLA        107546 non-null  float64
 7   Estado                 107518 non-null  object 
 8   Fecha de muerte        100460 non-null  object 
 9   Fecha de notificación  103754 non-null  object 
 10  Fecha recuperado       104971 non-null  object 
 11  FIS                    111146 non-null  object 
 12  Lugar_Atencion         153051 non-null  object 
 13  Depto                  153051 non-null  object 
 14  Tipo_Arreglado         153051 non-nu

In [24]:
#Creo la fecha del archivo
miFecha =data.apply(lambda x: datetime.date(int(x['año']), int(x['mes_archivo']), int(x['dia_archivo'])),axis=1)
data['Fecha_Archivo']= miFecha
data = data.drop(['mes_archivo', 'dia_archivo', 'año', 'source'], 1)

In [25]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 153051 entries, 0 to 153050
Data columns (total 17 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   ID de caso             153051 non-null  object 
 1   Ciudad de ubicación    153051 non-null  object 
 2   Edad                   153050 non-null  object 
 3   Sexo                   153051 non-null  object 
 4   País de procedencia    153040 non-null  object 
 5   Codigo DIVIPOLA        107546 non-null  float64
 6   Estado                 107518 non-null  object 
 7   Fecha de muerte        100460 non-null  object 
 8   Fecha de notificación  103754 non-null  object 
 9   Fecha recuperado       104971 non-null  object 
 10  FIS                    111146 non-null  object 
 11  Lugar_Atencion         153051 non-null  object 
 12  Depto                  153051 non-null  object 
 13  Tipo_Arreglado         153051 non-null  object 
 14  Fecha Diagnostico      153051 non-nu

In [26]:
#Casos Totales por día
data['Fecha_Archivo'].value_counts()

2020-05-06    8959
2020-05-05    8613
2020-05-04    7973
2020-05-03    7708
2020-05-02    7285
2020-05-01    6507
2020-04-30    6507
2020-04-29    6211
2020-04-28    5949
2020-04-27    5597
2020-04-26    5379
2020-04-25    5142
2020-04-24    4881
2020-04-23    4561
2020-04-22    4356
2020-04-21    4149
2020-04-20    3977
2020-04-19    3792
2020-04-18    3621
2020-04-17    3439
2020-04-16    3233
2020-04-14    2979
2020-04-15    2979
2020-04-13    2852
2020-04-12    2776
2020-04-11    2709
2020-04-10    2473
2020-04-09    2223
2020-04-08    2054
2020-04-07    1780
2020-04-06    1579
2020-04-05    1485
2020-04-03    1267
2020-04-02    1161
2020-04-01    1065
2020-03-31     906
2020-03-30     798
2020-03-29     702
2020-03-28     608
2020-03-26     491
2020-03-27     491
2020-03-25     470
2020-03-24     378
2020-03-23     306
2020-03-22     231
2020-03-21     196
2020-03-20     145
2020-03-19     108
Name: Fecha_Archivo, dtype: int64

In [27]:
data['Asintomático'] = data['FIS'] == 'Asintomático'
data['Asintomático'] = data['Asintomático'].replace({True:'Asintomático',False:''})
data['Asintomático'].value_counts()

                142543
Asintomático     10508
Name: Asintomático, dtype: int64

In [28]:
data['FIS'] = data['FIS'].replace({'Asintomático':""})
data['FIS'].value_counts()

                           10508
2020-03-20T00:00:00.000     3497
2020-04-01T00:00:00.000     3133
2020-03-19T00:00:00.000     2919
2020-03-21T00:00:00.000     2777
                           ...  
2020-05-03T00:00:00.000       34
2020-03-03T00:00:00.000       23
2020-02-28T00:00:00.000       19
2020-05-04T00:00:00.000        5
2020-05-05T00:00:00.000        1
Name: FIS, Length: 72, dtype: int64

In [29]:
## Funcion para convertir la edad a rango
def numtorange(edad):
    import numpy as np
    try:
        edad=int(edad)
    except:
        edad=edad
        
    if isinstance(edad, str) :
        if 'meses' in edad:
            rango='0 a 9'
        elif 'mes' in edad:
            rango='0 a 9'
        else:
            rango=edad
    else:
        if np.isnan(edad):
            rango=edad
        else:
            ini=round(edad/10)*10
            fin=ini+9
            rango=str(ini)+' a '+str(fin)
    return rango

In [30]:
## Convirtiendo a rango de edad
data['EdadRango']=data['Edad'].apply(lambda x: numtorange(x))

In [31]:
data.tail()

,ID de caso,Ciudad de ubicación,Edad,Sexo,País de procedencia,Codigo DIVIPOLA,Estado,Fecha de muerte,Fecha de notificación,Fecha recuperado,FIS,Lugar_Atencion,Depto,Tipo_Arreglado,Fecha Diagnostico,Fecha_Reporte_Web,Fecha_Archivo,Asintomático,EdadRango
153046,2219,Cali,62,M,Colombia,NaN,NaN,NaN,NaN,NaN,NaN,Hospital,VALLE DEL CAUCA,En Estudio,09/04/2020,NaN,2020-04-09,,60 a 69
153047,2220,Manizales,32,M,Colombia,NaN,NaN,NaN,NaN,NaN,NaN,Casa,VALLE DEL CAUCA,En Estudio,09/04/2020,NaN,2020-04-09,,30 a 39
153048,2221,Yopal,43,F,Colombia,NaN,NaN,NaN,NaN,NaN,NaN,Casa,CASANARE,En Estudio,09/04/2020,NaN,2020-04-09,,40 a 49
153049,2222,Espinal,44,M,Colombia,NaN,NaN,NaN,NaN,NaN,NaN,Casa,TOLIMA,En Estudio,09/04/2020,NaN,2020-04-09,,40 a 49
153050,2223,Ipiales,42,M,Colombia,NaN,NaN,NaN,NaN,NaN,NaN,Hospital,NARIÑO,En Estudio,09/04/2020,NaN,2020-04-09,,40 a 49


In [32]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 153051 entries, 0 to 153050
Data columns (total 19 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   ID de caso             153051 non-null  object 
 1   Ciudad de ubicación    153051 non-null  object 
 2   Edad                   153050 non-null  object 
 3   Sexo                   153051 non-null  object 
 4   País de procedencia    153040 non-null  object 
 5   Codigo DIVIPOLA        107546 non-null  float64
 6   Estado                 107518 non-null  object 
 7   Fecha de muerte        100460 non-null  object 
 8   Fecha de notificación  103754 non-null  object 
 9   Fecha recuperado       104971 non-null  object 
 10  FIS                    111146 non-null  object 
 11  Lugar_Atencion         153051 non-null  object 
 12  Depto                  153051 non-null  object 
 13  Tipo_Arreglado         153051 non-null  object 
 14  Fecha Diagnostico      153051 non-nu

In [35]:
data['Fecha Diagnostico'].unique()

array(['3/06/2020', '3/09/2020', '3/11/2020', '3/12/2020', '13/03/2020',
       '14/03/2020', '15/03/2020', '16/03/2020', '17/03/2020',
       '18/03/2020', '19/03/2020', '20/03/2020', '3/13/20', '3/14/20',
       '3/15/20', '3/16/20', '3/17/20', '3/18/20', '3/19/20', '3/20/20',
       '3/21/20', '3/22/20', '22/03/2020', '3/23/20', '23/03/2020',
       '6/03/2020', '9/03/2020', '11/03/2020', '12/03/2020', '21/03/2020',
       '24/03/2020', '27/03/2020', '25/03/2020', '26/03/2020',
       '28/03/2020', '29/03/2020', '30/03/2020', '31/03/2020',
       '1/04/2020', '6/04/2020', '21/04/2020', '3/04/2020', '4/04/2020',
       '2/04/2020', '8/04/2020', '7/04/2020', '23/04/2020', '9/04/2020',
       '13/04/2020', '15/04/2020', '5/04/2020', '24/04/2020',
       '11/04/2020', '18/04/2020', '17/04/2020', '25/04/2020',
       '10/04/2020', '14/04/2020', '12/04/2020', '16/04/2020',
       '19/04/2020', '20/04/2020', '22/04/2020', '6/03/20', '9/03/20',
       '11/03/20', '12/03/20', '13/03/20', '14

In [39]:
data[data['Fecha Diagnostico']=='43896']

,ID de caso,Ciudad de ubicación,Edad,Sexo,País de procedencia,Codigo DIVIPOLA,Estado,Fecha de muerte,Fecha de notificación,Fecha recuperado,FIS,Lugar_Atencion,Depto,Tipo_Arreglado,Fecha Diagnostico,Fecha_Reporte_Web,Fecha_Archivo,Asintomático,EdadRango


In [33]:
#data.to_csv(r'C:\Users\scamacho\Desktop\COVID-19\Historico de Casos Python\CSV_limpio\Historico_Covid.csv')